In [1]:
import os
import numpy as np
import pandas as pd
from scipy.fftpack import fft
from scipy.io import wavfile
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from python_speech_features import mfcc

In [2]:
ROOT_DIR = os.path.normpath(os.path.join(os.path.dirname(os.path.realpath('__file__'))))
DATA_DIR = os.path.join(ROOT_DIR, 'data_asr', 'data')
DATA_INFO = os.path.join(ROOT_DIR, 'data_asr', 'data_info')

In [3]:
window_len = 30
data_list = []

In [4]:
test_files = pd.read_csv(os.path.join(DATA_INFO, 'testing_list.txt'), sep = ' ', header = None)[0].tolist()
val_files = pd.read_csv(os.path.join(DATA_INFO, 'validation_list.txt'), sep = ' ', header = None)[0].tolist()

In [5]:
test_lab = [os.path.dirname(i) for i in test_files]
val_lab = [os.path.dirname(i) for i in val_files]

In [6]:
test_list = [os.path.join(DATA_DIR, f) for f in test_files if f.endswith('.wav')]
val_list = [os.path.join(DATA_DIR, f) for f in val_files if f.endswith('.wav')]

In [7]:
for root, dirs, files in os.walk(DATA_DIR):
    data_list += [root + '/'+ f for f in files if f.endswith('.wav')]

In [8]:
train_list = list(set(data_list) - set(test_list) - set(val_list))

In [9]:
train_lab = [os.path.basename(os.path.dirname(i)) for i in train_list]

In [10]:
lab = LabelEncoder()

train_encode = lab.fit_transform(train_lab)
y_encode = lab.fit_transform(train_encode)

val_encode = lab.fit_transform(val_lab)
yval_encode = lab.fit_transform(val_encode)

test_encode = lab.fit_transform(test_lab)
ytest_encode = lab.fit_transform(test_encode)

In [11]:
lab_binarizer = LabelBinarizer()
lab_binarizer.fit(range(len(np.unique(test_lab))))

ytrain = lab_binarizer.transform(y_encode)
yval = lab_binarizer.transform(yval_encode)
ytest = lab_binarizer.transform(ytest_encode)

In [12]:
samp_train, samp_val, samp_test, x_train, x_val, x_test = [], [], [], [], [], []

for i in train_list:
    samp_, x = wavfile.read(i)
    samp_train.append(samp_)
    x_train.append(x)
    
for i in val_list:
    samp_, x = wavfile.read(i)
    samp_val.append(samp_)
    x_val.append(x)
    
for i in test_list:
    samp_, x = wavfile.read(i)
    samp_test.append(samp_)
    x_test.append(x)

In [13]:
xtrain_clean, xval_feat, xtest_feat = [], [], []

for i in x_train:
    xtrain_clean.append(mfcc(i, winlen = 0.03, numcep = 40, nfilt = 40))